In [1]:
import pymysql.cursors
import pandas as pd

conn = pymysql.connect(host='localhost', 
                       user='root', 
                       password='root', 
                       db='zio', 
                       charset='utf8mb4', 
                       autocommit=True, 
                       cursorclass=pymysql.cursors.DictCursor)

def get_investors_volume() :
    cur = conn.cursor()
    sql = "select DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND from CRAWL_INVESTOR_DAILY_VOLUME where TYPE_CODE='02'"
    cur.execute(sql)
    result = cur.fetchall()
    cur.close()
    df = pd.DataFrame(result)
    df = df.set_index('DATEON')
    return df

def exec_query(sql) -> pd.DataFrame :
    cur = conn.cursor()
    cur.execute(sql)
    result = cur.fetchall()
    # print(result.count())
    cur.close()
    df = pd.DataFrame(result)
    return df

def end_price(sql = "select * from CRWAL_INVESTOR_STOCK_TYPE") :
    return exec_query(sql)

def get_daily_kosdaq() :
    sql = 'select * from CRAWL_MARKET_DAILY_DATA'
    return exec_query(sql)

def get_daily_investors_volume() :
    sql = "select DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND from CRAWL_INVESTOR_DAILY_VOLUME where TYPE_CODE='02'"
    return exec_query(sql)

# get_investors_volume()

#end_price()
get_daily_kosdaq()

# from pycaret.datasets import get_data
# get_data('insurance')

,TYPE_CODE,DATEON,CLOSING_PRICE,UP_DOWN_PER,VOLUME,VOLUME_AMT
0,KOSDAQ,1996.07.04,1018.40,1.30,235.0,5247.0
1,KOSDAQ,1996.07.05,1037.80,1.90,59.0,894.0
2,KOSDAQ,1996.07.06,1048.80,1.06,34.0,338.0
3,KOSDAQ,1996.07.08,1048.20,-0.06,39.0,822.0
4,KOSDAQ,1996.07.09,1060.10,1.14,45.0,589.0
...,...,...,...,...,...,...
7045,KOSDAQ,2024.08.14,776.83,1.56,1111020.0,7691490.0
7046,KOSDAQ,2024.08.16,786.33,1.22,1080400.0,8000420.0
7047,KOSDAQ,2024.08.19,777.47,-1.13,1135630.0,7653030.0
7048,KOSDAQ,2024.08.20,787.44,1.28,997056.0,7990590.0


In [2]:
investors_volume = get_daily_investors_volume()
# investors_volume = investors_volume.sort_value(by=['DATEON'], axis=0, ascending=True)
# investors_volume.reset_index()
investors_volume = investors_volume.set_index('DATEON')
investors_volume.sort_index(ascending=False, inplace=True)

daily_kosdaq = get_daily_kosdaq()
daily_kosdaq['DATEON'] = daily_kosdaq['DATEON'].str[2:]
daily_kosdaq.set_index('DATEON').drop(labels = ['TYPE_CODE'], axis=1)

import pandas as pd

merged = pd.merge(left=investors_volume, right=daily_kosdaq, how='inner', on='DATEON')

merged.set_index('DATEON')
merged.sort_index(ascending=True, inplace=True)
merged = merged.set_index('DATEON')
merged = merged.drop(labels = ['TYPE_CODE'], axis=1)
merged

,PERSONAL,FOREIGNER,COMPANY,FINANCE,INSURANCE,TOOSIN,BANK,ETC_FIN,GOV_FUND,ETC_FUND,CLOSING_PRICE,UP_DOWN_PER,VOLUME,VOLUME_AMT
DATEON,,,,,,,,,,,,,,
24.08.16,-1344.0,1010.0,659.0,9.0,22.0,427.0,-14.0,-43.0,258.0,-325.0,786.33,1.22,1080400.0,8000420.0
24.08.14,-807.0,828.0,8.0,-84.0,22.0,67.0,-2.0,-47.0,52.0,-29.0,776.83,1.56,1111020.0,7691490.0
24.08.13,2406.0,-1742.0,-689.0,2.0,-29.0,-408.0,0.0,-33.0,-220.0,25.0,764.86,-1.02,1170510.0,7661970.0
24.08.12,1716.0,-1550.0,-157.0,-99.0,20.0,112.0,-1.0,-67.0,-121.0,-10.0,772.72,1.08,885108.0,6850310.0
24.08.09,-1214.0,508.0,745.0,-202.0,57.0,874.0,4.0,-7.0,18.0,-39.0,764.43,2.57,843852.0,7392820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05.01.25,275.0,-76.0,-199.0,-24.0,-17.0,4.0,-34.0,-11.0,-117.0,0.0,467.29,-1.15,691510.0,1931120.0
05.01.24,321.0,-124.0,-196.0,-33.0,12.0,-48.0,22.0,-15.0,-135.0,0.0,472.75,2.63,603670.0,1756330.0
05.01.21,235.0,-177.0,-58.0,3.0,-17.0,-49.0,24.0,-2.0,-17.0,0.0,460.62,2.22,607820.0,1587980.0


In [3]:
def create_ex_columns(df , name: str) :
    # print(name + ' newly added ..')
    df[name + '_SUM_D5'] = df[name].rolling(5).sum().shift(1)
    df[name + '_AVG_D5'] = df[name].rolling(5).mean().shift(1)
    df[name + '_STD_D5'] = df[name].rolling(5).std().shift(1)
    df[name + '_SUM_D10'] = df[name].rolling(10).sum().shift(1)
    df[name + '_AVG_D10'] = df[name].rolling(10).mean().shift(1)
    df[name + '_STD_D10'] = df[name].rolling(10).std().shift(1)
    df[name + '_SUM_D20'] = df[name].rolling(20).sum().shift(1)
    df[name + '_AVG_D20'] = df[name].rolling(20).mean().shift(1)
    df[name + '_STD_D20'] = df[name].rolling(20).std().shift(1)

feature_cols = ['PERSONAL', 'FOREIGNER', 'COMPANY', 'FINANCE', 'INSURANCE', 'TOOSIN', 'BANK', 'ETC_FIN', 'GOV_FUND', 'ETC_FUND',
                'CLOSING_PRICE', 'UP_DOWN_PER', 'VOLUME', 'VOLUME_AMT']

for col in feature_cols:
    create_ex_columns(merged, col)
    
# merged['PRED_OUT'] = merged['CLOSING_PRICE'].shift(1)
merged['PRED_OUT_D3'] = merged['CLOSING_PRICE'].rolling(3).mean().shift(1)
# merged[['CLOSING_PRICE', 'PRED_OUT', 'PRED_OUT_D3']]
merged = merged.dropna()
merged.sort_index(ascending=False)
merged                          

/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_40567/4029696418.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name + '_STD_D20'] = df[name].rolling(20).std().shift(1)
/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_40567/4029696418.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name + '_SUM_D5'] = df[name].rolling(5).sum().shift(1)
/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_40567/4029696418.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

,PERSONAL,FOREIGNER,COMPANY,FINANCE,INSURANCE,TOOSIN,BANK,ETC_FIN,GOV_FUND,ETC_FUND,...,VOLUME_AMT_SUM_D5,VOLUME_AMT_AVG_D5,VOLUME_AMT_STD_D5,VOLUME_AMT_SUM_D10,VOLUME_AMT_AVG_D10,VOLUME_AMT_STD_D10,VOLUME_AMT_SUM_D20,VOLUME_AMT_AVG_D20,VOLUME_AMT_STD_D20,PRED_OUT_D3
DATEON,,,,,,,,,,,,,,,,,,,,,
24.07.18,54.0,823.0,-744.0,1004.0,-196.0,-1274.0,-8.0,-23.0,-246.0,-133.0,...,34711100.0,6942220.0,345526.059509,64778020.0,6477802.0,646478.406768,140874170.0,7043708.5,890586.053275,816.933333
24.07.17,543.0,-221.0,-289.0,983.0,-81.0,-608.0,-1.0,-45.0,-538.0,-32.0,...,34394690.0,6878938.0,293373.062022,65660260.0,6566026.0,646471.734936,139836740.0,6991837.0,861673.169791,820.386667
24.07.16,1617.0,-901.0,-603.0,277.0,-18.0,-518.0,5.0,-34.0,-314.0,-113.0,...,35352030.0,7070406.0,363831.460954,66489070.0,6648907.0,717782.101756,139695330.0,6984766.5,856192.718912,826.870000
24.07.15,-766.0,1083.0,-160.0,-41.0,-21.0,271.0,-2.0,-94.0,-275.0,-156.0,...,35444180.0,7088836.0,382581.752741,67656160.0,6765616.0,742407.434951,139462810.0,6973140.5,848053.158205,830.500000
24.07.12,-180.0,851.0,-545.0,-6.0,-95.0,-421.0,-1.0,-36.0,15.0,-126.0,...,35258620.0,7051724.0,438080.471523,68773520.0,6877352.0,573842.201806,139099100.0,6954955.0,854699.365487,840.633333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05.01.25,275.0,-76.0,-199.0,-24.0,-17.0,4.0,-34.0,-11.0,-117.0,0.0,...,9469240.0,1893848.0,257540.243205,16780760.0,1678076.0,296311.325437,35715190.0,1785759.5,275362.287455,467.823333
05.01.24,321.0,-124.0,-196.0,-33.0,12.0,-48.0,22.0,-15.0,-135.0,0.0,...,9747110.0,1949422.0,219867.780996,17121730.0,1712173.0,304572.167075,35893720.0,1794686.0,277118.558029,467.410000
05.01.21,235.0,-177.0,-58.0,3.0,-17.0,-49.0,24.0,-2.0,-17.0,0.0,...,9783470.0,1956694.0,210795.668077,17560600.0,1756060.0,271163.879355,35836330.0,1791816.5,277208.207695,469.510000


In [4]:
train = merged.sample(frac=0.95, random_state=786)
test = merged.drop(train.index)
# train.reset_index(inplace=True, drop=False)
# test.reset_index(inplace=True, drop=False)

# train.sort_index(ascending=False)
test.sort_index(ascending=False)

,PERSONAL,FOREIGNER,COMPANY,FINANCE,INSURANCE,TOOSIN,BANK,ETC_FIN,GOV_FUND,ETC_FUND,...,VOLUME_AMT_SUM_D5,VOLUME_AMT_AVG_D5,VOLUME_AMT_STD_D5,VOLUME_AMT_SUM_D10,VOLUME_AMT_AVG_D10,VOLUME_AMT_STD_D10,VOLUME_AMT_SUM_D20,VOLUME_AMT_AVG_D20,VOLUME_AMT_STD_D20,PRED_OUT_D3
DATEON,,,,,,,,,,,,,,,,,,,,,
24.07.11,1950.0,-47.0,-1660.0,-232.0,16.0,-1192.0,-3.0,-40.0,-210.0,-243.0,...,35664620.0,7132924.0,423993.192787,70375720.0,7037572.0,378234.786043,138941780.0,6947089.0,8.511742e+05,847.620000
24.07.03,-526.0,1866.0,-1280.0,-604.0,-5.0,-384.0,-33.0,-73.0,-181.0,-60.0,...,43968260.0,8793652.0,545104.209367,80839350.0,8083935.0,924904.370435,146499610.0,7324980.5,1.099775e+06,849.190000
24.07.01,-1109.0,104.0,1167.0,960.0,43.0,241.0,-12.0,-37.0,-28.0,-162.0,...,44556360.0,8911272.0,704341.580961,83497350.0,8349735.0,964412.137168,151153510.0,7557675.5,1.167059e+06,835.606667
24.06.24,2428.0,56.0,-1750.0,-527.0,-30.0,-1027.0,-12.0,-104.0,-50.0,-735.0,...,37145380.0,7429076.0,375080.566585,81701740.0,8170174.0,945127.412483,156086910.0,7804345.5,9.106804e+05,840.920000
24.04.24,-2444.0,1826.0,727.0,-16.0,82.0,577.0,18.0,-16.0,83.0,-109.0,...,39638790.0,7927758.0,206433.385672,84241070.0,8424107.0,751409.871036,177659700.0,8882985.0,8.336721e+05,859.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
05.12.19,428.0,-196.0,-233.0,-199.0,0.0,43.0,-29.0,15.0,-63.0,0.0,...,13979220.0,2795844.0,339234.898455,26590670.0,2659067.0,435708.987897,57053080.0,2852654.0,4.350558e+05,711.856667
05.09.26,-70.0,-124.0,194.0,-12.0,-14.0,102.0,29.0,-6.0,95.0,0.0,...,9006610.0,1801322.0,216931.172771,20212110.0,2021211.0,283907.235259,42438770.0,2121938.5,2.575488e+05,561.653333
05.06.07,76.0,-2.0,-74.0,-48.0,5.0,-9.0,14.0,1.0,-36.0,0.0,...,11358360.0,2271672.0,460674.735437,20214760.0,2021476.0,411743.414444,37308680.0,1865434.0,3.517235e+05,486.523333


In [5]:
from pycaret.regression import *

s = setup(data = train, target='PRED_OUT_D3', session_id=123, normalize=True, normalize_method='zscore')

ImportError: cannot import name 'interp' from 'scipy' (/opt/miniconda3/envs/myFirstPyCrawlNew/lib/python3.9/site-packages/scipy/__init__.py)

In [ ]:
best = compare_models(include=['et', 'rf', 'lightgbm'])
evaluate_model(best)
pred_holdout = predict_model(best)
interpret_model(best)